In [7]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

In [8]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)
data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
767,6.8,0.200,0.36,1.6,0.028,7.0,46.0,0.99175,3.21,0.60,10.9,6
2786,8.0,0.300,0.36,11.0,0.034,8.0,70.0,0.99354,3.05,0.41,12.2,6
2598,7.8,0.280,0.31,2.1,0.046,28.0,208.0,0.99434,3.23,0.64,9.8,5
4027,6.4,0.260,0.26,1.1,0.052,22.0,176.0,0.99304,3.09,0.54,9.3,5
594,6.4,0.475,0.06,1.0,0.030,9.0,131.0,0.99040,2.97,0.29,10.8,5
2980,6.1,0.380,0.42,5.0,0.016,31.0,113.0,0.99007,3.15,0.31,12.4,7
4095,6.6,0.410,0.27,10.7,0.110,20.0,103.0,0.99672,3.08,0.41,9.0,6
182,6.8,0.280,0.40,22.0,0.048,48.0,167.0,1.00100,2.93,0.50,8.7,5
491,7.1,0.230,0.39,1.6,0.032,12.0,65.0,0.98980,3.25,0.40,12.7,7
478,7.1,0.550,0.13,1.7,0.073,21.0,165.0,0.99400,2.97,0.58,9.2,6


In [9]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 0.0.0.0 --port 5000
remote_server_uri = "http://127.0.0.1:5000/" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [10]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000/'

In [6]:
exp_name="ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/08/24 02:19:17 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [12]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [13]:
train(0.5,0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772677
Save to: ./mlruns/1/abeef6aacb3b424b8c6363ff09bc589e/artifacts


In [14]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062341
  MAE: 0.6155290394093895
  R2: 0.20224631822892103
Save to: ./mlruns/1/333b31370acc4a6cb55b1100a3bc4512/artifacts


In [15]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: ./mlruns/1/ebe8f364c37b47418980f5f7fa971262/artifacts


In [16]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='file:///mnt/d/New%20folder/MLFlows/mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='file:///mnt/d/New%20folder/MLFlows/mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>]


In [17]:
# get the run
_run = client.get_run(run_id="4dad29736b9a432ebdc11f20e3d7f5fc")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6278761410160693,
 'r2': 0.12678721972772666,
 'rmse': 0.8222428497595401}, params={'alpha': '0.5', 'l1_ratio': '0.5'}, tags={'mlflow.log-model.history': '[{"run_id": "4dad29736b9a432ebdc11f20e3d7f5fc", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-08-24 09:21:25.797199", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"loader_module": "mlflow.sklearn", '
                             '"python_version": "3.6.8", "env": "conda.yaml"}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "0.21.3", '
                             '"serialization_format": "cloudpickle"}}, '
                             '"model_uuid": '
                             '"293a5854a7c64ff692c00b3a35b2ab58"}]',
 'mlflow.source.git.commit': 'c4be636b7936e0ff1b2ccb91

In [18]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)